In [3]:
import pandas as pd
import numpy as np

In [5]:
#先指定df
df = pd.read_excel("天數測試.xlsx")
df.tail(10)

,Series Code,Line No.,Starting Date,Starting No.,Ending No.,Warning No.,Increment-by No.,Last No. Used,Open,Last Date Used
1311,UPC,40000,2024-03-13,81016214000,81016214999,NaN,1,81016214108,Yes,2024-09-27
1312,US-,10000,2008-12-01,US-00001,US-99999,NaN,0,NaN,Yes,NaT
1313,VERSION,10000,2019-08-01,1,99999,NaN,1,1,Yes,2019-09-30
1314,WC,10000,2019-08-01,WC0001,WC9999,NaN,1,WC0014,Yes,2021-09-27
1315,WO,10000,2019-08-01,WO1900001,WO1999999,NaN,1,WO1900034,Yes,2019-12-17
1316,WO,20000,2020-01-01,WO2000001,WO2099999,NaN,1,WO2000082,Yes,2020-12-26
1317,WO,30000,2021-01-01,WO2100001,WO2199999,NaN,1,WO2100080,Yes,2021-12-23
1318,WO,40000,2022-01-01,WO2200001,WO2299999,NaN,1,WO2200169,Yes,2022-12-19
1319,WO,50000,2023-01-01,WO2300001,WO2399999,NaN,1,WO2300090,Yes,2023-12-20
1320,WO,60000,2024-01-01,WO2400001,WO2499999,NaN,1,WO2400002,Yes,2024-02-01


In [26]:
#函數find_code，尋找Data裡面的code
def find_code(df,input_code):
    for code in range(len(df)):
        if input_code == df["Series Code"][code]:
            print(f"{input_code} found.")
            return input_code
    print(f"Sorry, {input_code} is not found.")

find_code(df,"WO")

WO found.


'WO'

In [92]:
#函數date_format測試
series_code = "WO"
wo_code = date_format(df,"WO")
print(wo_code)

generate_series(df,series_code,wo_code)

yearly


(['WO'],
 [70000],
 ['WO2400001-250101-0001'],
 ['WO2400001-250101-9999'],
 ['2025/01/01'])

In [53]:
#定義date_format
def date_format(df,series_code):
    interval = ""
    # 過濾指定 series code 的數據

    #series_code = "WO"#測試用

    test_series = df["Series Code"] == series_code

    # 找到最大的 Line No
    max_line_no = df[test_series]["Line No."].max()
    result = df[test_series & (df["Line No."] == max_line_no)]
    # print(result)
    starting_date = pd.to_datetime(result["Starting Date"].values[0])
    previous_date =pd.to_datetime(df.iloc[result.index-1,2].values[0])
    time_difference = starting_date - previous_date
    time_difference = time_difference.days 

    if time_difference == 365 or time_difference == 366:
        interval = "yearly"
    elif time_difference == 31 or time_difference == 30:
        interval = "monthly"
    elif time_difference == 1:
        interval = "daily"
    else:
        print("Unrecognizable!")
    # print(interval)
    return interval
    # print(result.index)
    # print(starting_date)
    # print(previous_date)

    # print(type(time_difference))

#函數date_format測試
a = date_format(df,"PORP")
print(a)
print(type(a))

yearly
<class 'str'>


In [55]:
def generate_series(df, series_code, time_interval):
    # 過濾指定 series code 的數據
    test_series = df["Series Code"] == series_code

    # 找到最大的 Line No
    max_line_no = df[test_series]["Line No."].max()
    result = df[test_series & (df["Line No."] == max_line_no)]

    # 建立新的 series code 和 line no 列表
    series_list = []
    line_list = []
    date_list = []
    
    starting_date = result["Starting Date"].values[0]
    starting_date = pd.to_datetime(starting_date)
    
    if time_interval == "monthly":
        # 按月份生成日期
        for i in range(1, 13):
            new_date = starting_date + pd.DateOffset(months=i)
            date_list.append(new_date)
            series_list.append(series_code)
            max_line_no += 10000
            line_list.append(max_line_no)
    elif time_interval == "daily":
        # 按天數生成 365 天日期
        for i in range(1, 366):
            new_date = starting_date + pd.DateOffset(days=i)
            date_list.append(new_date)
            series_list.append(series_code)
            max_line_no += 10000
            line_list.append(max_line_no)
    elif time_interval == "yearly":
            new_date = starting_date + pd.DateOffset(years=1)
            date_list.append(new_date)
            series_list.append(series_code)
            max_line_no += 10000
            line_list.append(max_line_no)
        
    else:
        print("Time format is unrecognizable!")
    
    # 格式化日期
    formatted_date_list = [date.strftime('%Y/%m/%d') for date in date_list]

    # 獲取最後的 Starting No
    last_starting_no = result["Starting No."].values[0]
    prefix = str(last_starting_no).split("-")[0] + "-"

    # 生成新的 Starting No 和 Ending No
    starting_list = []
    ending_list = []

    for date in range(len(formatted_date_list)):
        year_2 = formatted_date_list[date][2:4]
        month_2 = formatted_date_list[date][5:7]
        date_2 = formatted_date_list[date][8:]

        if int(month_2) == 10:
            month_2 = "A"
        elif int(month_2) == 11:
            month_2 = "B"
        elif int(month_2) == 12:
            month_2 = "C"

        new_starting_no = prefix + year_2 + month_2 + date_2 + "-0001"
        starting_list.append(new_starting_no)
        
        if any(char in new_starting_no for char in ['A', 'B', 'C']):
            new_ending_no = new_starting_no[:-4] + "9999"
        else:
            new_ending_no = new_starting_no[:-5] + "-9999"
        
        ending_list.append(new_ending_no)

    # 輸出結果
    return series_list, line_list, starting_list, ending_list, formatted_date_list

# 使用範例
df = pd.read_excel("天數測試.xlsx")
series_code = "SOR"
series_list, line_list, starting_list, ending_list, formatted_date_list = generate_series(df, series_code, time_interval="monthly")


In [ ]:
input_code = input("Please enter your series code:")
if_search = find_code(df,input_code)
print(f"If search:{if_search}")

new_number = generate_series(df,input_code,if_search)


In [67]:
start_search = True
start_generation = True

while start_search:
    input_code = input("Please enter your series code:")
    if_search = find_code(df,input_code)
    if len(if_search) == 0:
        start_search = True
    else:
        start_search = False

while start_generation:
    time_interval = date_format(df,input_code)
    new_number = generate_series(df,input_code,time_interval)
    print(new_number[0],type(new_number[0]))
    
    break
    # print(1)
    # break

print(new_number)


Please enter your series code: WO


WO found.
['WO'] <class 'list'>
(['WO'], [70000], ['WO2400001-250101-0001'], ['WO2400001-250101-9999'], ['2025/01/01'])


In [69]:
print(new_number)

(['WO'], [70000], ['WO2400001-250101-0001'], ['WO2400001-250101-9999'], ['2025/01/01'])


In [94]:
print(generate_series(df, series_code, time_interval="monthly"))

(['WO', 'WO', 'WO', 'WO', 'WO', 'WO', 'WO', 'WO', 'WO', 'WO', 'WO', 'WO'], [70000, 80000, 90000, 100000, 110000, 120000, 130000, 140000, 150000, 160000, 170000, 180000], ['WO2400001-240201-0001', 'WO2400001-240301-0001', 'WO2400001-240401-0001', 'WO2400001-240501-0001', 'WO2400001-240601-0001', 'WO2400001-240701-0001', 'WO2400001-240801-0001', 'WO2400001-240901-0001', 'WO2400001-24A01-0001', 'WO2400001-24B01-0001', 'WO2400001-24C01-0001', 'WO2400001-250101-0001'], ['WO2400001-240201-9999', 'WO2400001-240301-9999', 'WO2400001-240401-9999', 'WO2400001-240501-9999', 'WO2400001-240601-9999', 'WO2400001-240701-9999', 'WO2400001-240801-9999', 'WO2400001-240901-9999', 'WO2400001-24A01-9999', 'WO2400001-24B01-9999', 'WO2400001-24C01-9999', 'WO2400001-250101-9999'], ['2024/02/01', '2024/03/01', '2024/04/01', '2024/05/01', '2024/06/01', '2024/07/01', '2024/08/01', '2024/09/01', '2024/10/01', '2024/11/01', '2024/12/01', '2025/01/01'])


In [17]:
df = pd.read_excel("天數測試.xlsx")
series_code = "LOT-NO"
series_list, line_list, starting_list, ending_list, formatted_date_list = generate_series(df, series_code, time_interval="monthly")
print(generate_series(df, series_code, time_interval="daily"))

(['LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO', 'LOT-NO'